In [1]:
import pandas as pd

import sys
sys.path.insert(1, '../../../scripts/')
from s3_support import *

# Users

In [23]:
q = "select id, uuid, org, name, timestamp 'epoch' + created_at * interval '1 second' as created_at, user_type, total_forms, donation_forms, status, p2p_events, political_forms from users order by created_at desc"
df_users = redshift_query_read(q, schema='secure')

In [87]:
df_users.head(3)

,id,uuid,org,name,created_at,user_type,total_forms,donation_forms,status,p2p_events,political_forms,created_at_month
0,1414439,11478,443788,Laurie Fasano,2020-03-29 13:33:14,Admin,5.0,1.0,None,2.0,0.0,2020-03
1,1414328,11239,None,Eimme Deogracias,2020-03-28 14:49:51,User,NaN,NaN,None,NaN,NaN,2020-03
2,1414327,11378,None,Corie Jackon,2020-03-28 14:48:33,User,NaN,NaN,None,NaN,NaN,2020-03


In [26]:
df_users['created_at_month'] = pd.to_datetime(df_users['created_at']).dt.to_period('M')
df_users[['id', 'created_at_month']].head()

,id,created_at_month
0,1414439,2020-03
1,1414328,2020-03
2,1414327,2020-03
3,1414319,2020-03
4,1414303,2020-03


In [30]:
created_monthly = df_users.groupby('created_at_month')['id'].count().reset_index()
created_monthly.columns = ['month', 'users_created']
created_monthly.tail(12)

,month,users_created
89,2019-04,168
90,2019-05,152
91,2019-06,177
92,2019-07,182
93,2019-08,234
94,2019-09,259
95,2019-10,301
96,2019-11,202
97,2019-12,199
98,2020-01,367


In [39]:
growth_df = created_monthly.tail(12)
growth_df['diff'] = growth_df['users_created'].diff() / growth_df['users_created']
growth_df['diff'].mean()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.014172427122372938

In [29]:
df_users[['id', 'org']].groupby('org')['id'].count().mean()

2.537668798862829

# Logins

In [78]:
q = '''select 
            user_id,
            users.org, 
            date_trunc('day', original_timestamp) as date 
        from login 
            left join users on login.user_id=users._id
        order by date desc'''
df_logins = redshift_query_read(q, schema='secure')

In [79]:
df_logins.head()

,user_id,org,date
0,1413807,445345,2020-03-30
1,1397225,445065,2020-03-30
2,1320461,444298,2020-03-30
3,71123,39950,2020-03-30
4,858340,380931,2020-03-30


In [80]:
df_logins['month'] = pd.to_datetime(df_logins['date']).dt.to_period('M')

In [86]:
df_logins.groupby('month')['user_id'].count().reset_index().head(12)

,month,user_id
0,2019-06,2
1,2019-10,3
2,2020-01,9675
3,2020-02,42306
4,2020-03,39317


In [83]:
df_logins.groupby('user_id')['user_id'].count().mean()

18.06906788046705

In [84]:
daily_logins = df_logins.groupby(['date'])['user_id'].count().reset_index().tail(60)
daily_logins['trailing'] = daily_logins['user_id'].rolling(window=7).mean()
daily_logins.head(15)

,date,user_id,trailing
11,2020-01-31,1699,NaN
12,2020-02-01,331,NaN
13,2020-02-02,272,NaN
14,2020-02-03,2119,NaN
15,2020-02-04,2179,NaN
16,2020-02-05,1995,NaN
17,2020-02-06,1962,1508.142857
18,2020-02-07,1622,1497.142857
19,2020-02-08,284,1490.428571
20,2020-02-09,249,1487.142857


In [85]:
logins_daily_last_60_days = df_logins.groupby(['date'])['user_id'].count().reset_index().tail(60)
logins_daily_last_60_days['trailing'] = logins_daily_last_60_days['user_id'].rolling(window=7).mean()

logins_daily_last_60_days.head()

,date,user_id,trailing
11,2020-01-31,1699,NaN
12,2020-02-01,331,NaN
13,2020-02-02,272,NaN
14,2020-02-03,2119,NaN
15,2020-02-04,2179,NaN


# Integrations

In [9]:
q = "select count(id) as activations, date_trunc('month', original_timestamp) as month from activated_integration group by month order by month desc"
redshift_query_read(q, schema='secure').head()

,activations,month
0,63,2020-03-01
1,90,2020-02-01
2,28,2020-01-01


In [10]:
q = "select count(id) as activations, integration, date_trunc('month', original_timestamp) as month from activated_integration group by integration, month order by month desc"
redshift_query_read(q, schema='secure').head()

,activations,integration,month
0,5,donorperfect,2020-03-01
1,14,bcrm_ja,2020-03-01
2,6,quickbooks,2020-03-01
3,11,salesforceexport,2020-03-01
4,1,raisersedge,2020-03-01


In [11]:
q = "select count(id) as activations, integration from activated_integration group by integration"
redshift_query_read(q, schema='secure').head()

,activations,integration
0,14,quickbooks
1,34,bcrm_ja
2,10,constantcontact
3,14,donorperfect
4,2,neon


In [12]:
q = '''select
    count(distinct(users.org)) as orgs,
    date_trunc('month', original_timestamp) as month
from activated_integration
    left join users on activated_integration.uuid=users.uuid
group by month
order by month desc'''
redshift_query_read(q, schema='secure').head()

,orgs,month
0,49,2020-03-01
1,58,2020-02-01
2,24,2020-01-01


# Page views

In [13]:
q = "select count(id) as pageviews, date_trunc('month', original_timestamp) as month, context_user_agent as useragent from pages group by month, useragent order by month desc"
redshift_query_read(q, schema='secure').head()

,pageviews,month,useragent
0,581,2020-03-01,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...
1,6148,2020-03-01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,146,2020-03-01,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5...
3,41594,2020-03-01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,10880,2020-03-01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...


In [89]:
q = "select count(id) as pageviews, date_trunc('month', original_timestamp) as month from pages group by month order by month desc"
redshift_query_read(q, schema='secure').head()

,pageviews,month
0,338795,2020-03-01
1,376121,2020-02-01
2,85990,2020-01-01
3,9,2019-12-01
4,1,2019-11-01


In [95]:
q = "select count(id) as pageviews, title, date_trunc('month', original_timestamp) as month from pages group by title, month order by month desc"
df_mp = redshift_query_read(q, schema='secure')

In [97]:
def normalize_title(title):
    if 'Transaction' in title:
        return 'Transaction'
    else:
        return title.replace(' - Qgiv', '')
df_mp['title'] = df_mp['title'].apply(normalize_title)
df_mp['title'].value_counts()

Transaction                                                53591
Form Dashboard                                                 7
My Qgiv                                                        5
Reporting                                                      5
Dashboard                                                      5
Receipts                                                       4
Givi Settings                                                  4
Batches & Settlements                                          4
Users                                                          4
Virtual Terminal                                               4
Event Dashboard                                                4
Text-to-Donate Campaigns                                       3
4 tickets to Celtic Journey with the Nashville Symphony        3
Item Categories                                                3
Items                                                          3
Donor Summary Compose    

# CMS

In [15]:
q = "select count(id) as page_updates, date_trunc('month', original_timestamp) as month from saved_page group by month order by month desc"
redshift_query_read(q, schema='secure').head()

,page_updates,month
0,11137,2020-03-01
1,9323,2020-02-01
2,2112,2020-01-01


In [16]:
q = '''select
    count(distinct(users.org)) as orgs,
    date_trunc('month', original_timestamp) as month
from saved_page
    left join users on saved_page.uuid=users.uuid
group by month
order by month desc'''
redshift_query_read(q, schema='secure').head()

,orgs,month
0,1810,2020-03-01
1,1811,2020-02-01
2,619,2020-01-01


# Widgets

In [22]:
q = "select count(id) as widgets_created, date_trunc('month', original_timestamp) as month from created_widget group by month order by month desc"
widgets = redshift_query_read(q, schema='secure')

q = "select count(id) as widgets_deleted, date_trunc('month', original_timestamp) as month from deleted_widget group by month order by month desc"
widgets_del = redshift_query_read(q, schema='secure')

q = "select count(id) as widgets_added, date_trunc('month', original_timestamp) as month from added_widget group by month order by month desc;"
widgets_add = redshift_query_read(q, schema="secure")

q = "select count(id) as widgets_updated, date_trunc('month', original_timestamp) as month from updated_widget group by month order by month desc;"
widgets_upd = redshift_query_read(q, schema="secure")

widgets = widgets.merge(widgets_del, on='month').merge(widgets_add, on="month").merge(widgets_upd, on="month")
widgets

,widgets_created,month,widgets_deleted,widgets_added,widgets_updated
0,43,2020-04-01,359,333,1764
1,287,2020-03-01,2258,2259,10286
2,206,2020-02-01,1518,1531,7505
3,39,2020-01-01,370,345,1804


In [18]:
q = '''select
    count(distinct(users.org)) as orgs,
    date_trunc('month', original_timestamp) as month
from created_widget
    left join users on created_widget.uuid=users.uuid
group by month
order by month desc'''
redshift_query_read(q, schema='secure').head()

,orgs,month
0,175,2020-03-01
1,145,2020-02-01
2,30,2020-01-01


# Integrations interest

In [5]:
tables = [
    'interested_in_blackbaud_crm_ja_integration',
    'interested_in_bloomerang_integration',
    'interested_in_campaign_monitor_integration',
    'interested_in_church_community_builder_integration',
    'interested_in_clear_view_crm_integration',
    'interested_in_constant_contact_integration',
    'interested_in_donor_perfect_integration',
    'interested_in_emma_integration',
    'interested_in_fellowship_one_integration',
    'interested_in_kindful_integration',
    'interested_in_mailchimp_integration',
    'interested_in_match_maker_integration',
    'interested_in_neon_crm_integration',
    'interested_in_offline_for_qgiv_vt',
    'interested_in_quick_books_online_integration',
    'interested_in_raiser_s_edge_import_omatic_integration',
    'interested_in_raiser_s_edge_importacular_integration',
    'interested_in_salesforce_integration',
    'interested_in_zapier_integration'
]
q = "select count(id) as {}, date_trunc('day', original_timestamp) as date from {} group by date"

In [17]:
df = None

for t in tables:
    t_name = t.replace('interested_in_', '').replace('_integration', '')
    if df is None:
        df = redshift_query_read(q.format(t_name, t), schema="secure")
    else:
        df = df.merge(redshift_query_read(q.format(t_name, t), schema="secure"), on="date", how="outer")

In [18]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', ascending=False, inplace=True)
df = df.fillna(0)

In [19]:
non_date_cols = [c for c in df.columns if c != 'date']
df = df[['date'] + non_date_cols]

In [20]:
df.head()

,date,blackbaud_crm_ja,bloomerang,campaign_monitor,church_community_builder,clear_view_crm,constant_contact,donor_perfect,emma,fellowship_one,kindful,mailchimp,match_maker,neon_crm,offline_for_qgiv_vt,quick_books_online,raiser_s_edge_import_omatic,raiser_s_edge_importacular,salesforce,zapier
30,2020-04-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
36,2020-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
29,2020-03-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0
37,2020-03-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
27,2020-03-25,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


# reporting filters

In [30]:
q = "select count(id) as advanced_filtering, date_trunc('month', original_timestamp) as month from applied_advanced_reporting_filter group by month order by month desc"
adv_filtering = redshift_query_read(q, schema="secure")

q = "select count(id) as basic_filtering, date_trunc('month', original_timestamp) as month from applied_basic_reporting_filter group by month order by month desc"
basic_filtering = redshift_query_read(q, schema="secure")

q = "select count(id) as report_downloads, date_trunc('month', original_timestamp) as month from downloaded_report group by month order by month desc"
report_downloads = redshift_query_read(q, schema="secure")

In [31]:
adv_filtering.merge(basic_filtering, on="month").merge(report_downloads, on="month")

,advanced_filtering,month,basic_filtering,report_downloads
0,5482,2020-04-01,12837,1030
1,42066,2020-03-01,83302,6525
2,41388,2020-02-01,53305,6181
3,8463,2020-01-01,13193,1483


# goals

In [65]:
q = "select count(id) as goals_deleted, date_trunc('month', original_timestamp) as month from goals_deleted_item group by month order by month desc"
goals_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as goals_saved, date_trunc('month', original_timestamp) as month from goals_saved_item_settings group by month order by month desc"
goals_saved = redshift_query_read(q, schema="secure")

In [66]:
goals_saved.merge(goals_deleted, on="month", how="outer").fillna(0)

,goals_saved,month,goals_deleted
0,76,2020-04-01,6
1,74,2020-03-01,10


# fundhub builder

In [68]:
q = "select count(id) as fundhub_builder_accessed, date_trunc('month', original_timestamp) as month from accessed_fund_hub_builder group by month order by month desc"
fundhub_accessed = redshift_query_read(q, schema="secure")
q = "select count(id) as event_builder_accessed, date_trunc('month', original_timestamp) as month from accessed_event_builder group by month order by month desc"
eventbuilder_accessed = redshift_query_read(q, schema="secure")
q = "select count(id) as form_builder_accessed, date_trunc('month', original_timestamp) as month from accessed_form_builder group by month order by month desc"
formbuilder_accessed = redshift_query_read(q, schema="secure")

In [72]:
cols = ['month', 'fundhub_builder_accessed', 'event_builder_accessed', 'form_builder_accessed']
fundhub_accessed.merge(eventbuilder_accessed, on="month", how="outer").merge(formbuilder_accessed, on="month")[cols].sort_values("month", ascending=True)

,month,fundhub_builder_accessed,event_builder_accessed,form_builder_accessed
3,2020-01-01,40,912,624
2,2020-02-01,203,3775,3038
1,2020-03-01,219,4256,3734
0,2020-04-01,50,1752,1475


# categories

In [17]:
q = "select count(id) as categories_deleted, date_trunc('month', original_timestamp) as month from categories_deleted_item group by month order by month desc"
categories_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as categories_saved, date_trunc('month', original_timestamp) as month from categories_saved_item_settings group by month order by month desc"
categories_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as categories_started_edit, date_trunc('month', original_timestamp) as month from categories_started_item_edit group by month order by month desc"
categories_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as categories_status_updated, date_trunc('month', original_timestamp) as month from categories_updated_item_status group by month order by month desc"
categories_status_edit = redshift_query_read(q, schema="secure")

In [19]:
len(categories_deleted), len(categories_saved), len(categories_started_edit), len(categories_status_edit)

(2, 2, 2, 2)

In [75]:
cols = ['month', 'categories_started_edit', 'categories_saved', 'categories_status_updated', 'categories_deleted']
categories_deleted.merge(categories_saved, on='month', how="outer").merge(categories_started_edit, on='month', how="outer").merge(categories_status_edit, on='month', how="outer")[cols].sort_values('month', ascending=True)

,month,categories_started_edit,categories_saved,categories_status_updated,categories_deleted
1,2020-03-01,287,250,70,59
0,2020-04-01,161,150,59,27


# cloned form

In [20]:
q = "select count(id) as forms_cloned, date_trunc('month', original_timestamp) as month from cloned_form group by month order by month desc"
forms_cloned = redshift_query_read(q, schema="secure")

In [21]:
forms_cloned

,forms_cloned,month
0,240,2020-04-01
1,638,2020-03-01
2,591,2020-02-01
3,127,2020-01-01


# badges

In [25]:
q = "select count(id) as custom_badges_created, date_trunc('month', original_timestamp) as month from created_custom_badge group by month order by month desc"
badges_created = redshift_query_read(q, schema="secure")

In [26]:
badges_created

,custom_badges_created,month
0,9,2020-04-01
1,27,2020-03-01
2,99,2020-02-01
3,4,2020-01-01


# custom fields

In [29]:
q = "select count(id) as custom_fields_deleted, date_trunc('month', original_timestamp) as month from custom_fields_deleted_item group by month order by month desc"
customfields_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as custom_fields_saved, date_trunc('month', original_timestamp) as month from custom_fields_saved_item_settings group by month order by month desc"
customfields_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as custom_fields_started_edit, date_trunc('month', original_timestamp) as month from custom_fields_started_item_edit group by month order by month desc"
customfields_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as custom_fields_updated_status, date_trunc('month', original_timestamp) as month from custom_fields_updated_item_status group by month order by month desc"
customfields_updated_status = redshift_query_read(q, schema="secure")

In [76]:
cols = ["month", "custom_fields_deleted", "custom_fields_saved", "custom_fields_started_edit", "custom_fields_updated_status"]
customfields_deleted.merge(customfields_saved, on="month", how="outer").merge(customfields_started_edit, on="month", how="outer").merge(customfields_updated_status, on="month", how="outer")[cols].sort_values('month', ascending=True)

,month,custom_fields_deleted,custom_fields_saved,custom_fields_started_edit,custom_fields_updated_status
1,2020-03-01,63,309,289,116
0,2020-04-01,41,256,279,85


# receipts

In [45]:
q = "select count(id) as custom_receipts_edited, date_trunc('month', original_timestamp) as month from edited_custom_receipt group by month order by month desc"
custom_receipts_edited = redshift_query_read(q, schema="secure")
q = "select count(id) as receipts_deleted, date_trunc('month', original_timestamp) as month from receipts_deleted_item group by month order by month desc"
receipts_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as receipts_saved, date_trunc('month', original_timestamp) as month from receipts_saved_item_settings group by month order by month desc"
receipts_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as receipts_started_edit, date_trunc('month', original_timestamp) as month from receipts_started_item_edit group by month order by month desc"
receipts_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as receipts_updated_status, date_trunc('month', original_timestamp) as month from receipts_updated_item_status group by month order by month desc"
receipts_updated_status = redshift_query_read(q, schema="secure")

In [46]:
custom_receipts_edited.merge(receipts_deleted, on="month").merge(receipts_saved, on="month").merge(receipts_started_edit, on="month").merge(receipts_updated_status, on="month")

,custom_receipts_edited,month,receipts_deleted,receipts_saved,receipts_started_edit,receipts_updated_status
0,765,2020-04-01,1,749,1441,135
1,1958,2020-03-01,11,1169,2106,198


# pledges

In [36]:
q = "select count(id) as pledges_saved, date_trunc('month', original_timestamp) as month from pledges_saved_item_settings group by month order by month desc"
pledges_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as pledges_started_edit, date_trunc('month', original_timestamp) as month from pledges_started_item_edit group by month order by month desc"
pledges_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as pledges_updated_status, date_trunc('month', original_timestamp) as month from pledges_updated_item_status group by month order by month desc"
pledges_updated_status = redshift_query_read(q, schema="secure")

In [77]:
cols = ["month", "pledges_saved", "pledges_started_edit", "pledges_updated_status"]
pledges_saved.merge(pledges_started_edit, on="month", how="outer").merge(pledges_updated_status, on="month", how="outer")[cols].sort_values('month', ascending=True)

,month,pledges_saved,pledges_started_edit,pledges_updated_status
1,2020-03-01,21,23,8
0,2020-04-01,26,24,14


# promo

In [40]:
q = "select count(id) as promo_deleted, date_trunc('month', original_timestamp) as month from promo_codes_deleted_item group by month order by month desc"
promo_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as promo_saved, date_trunc('month', original_timestamp) as month from promo_codes_saved_item_settings group by month order by month desc"
promo_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as promo_started_edit, date_trunc('month', original_timestamp) as month from promo_codes_started_item_edit group by month order by month desc"
promo_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as promo_status_updated, date_trunc('month', original_timestamp) as month from promo_codes_updated_item_status group by month order by month desc"
promo_updated_status = redshift_query_read(q, schema="secure")

In [78]:
cols = ["month", "promo_deleted", "promo_saved", "promo_started_edit", "promo_status_updated"]
promo_deleted.merge(promo_saved, on="month").merge(promo_started_edit, on="month").merge(promo_updated_status, on="month")[cols].sort_values('month', ascending=True)

,month,promo_deleted,promo_saved,promo_started_edit,promo_status_updated
1,2020-03-01,152,71,43,10
0,2020-04-01,1,28,30,10


# sponsors

In [54]:
q = "select count(id) as sponsors_started_edit, date_trunc('month', original_timestamp) as month from sponsors_started_item_edit group by month order by month desc"
sponsors_started_edit = redshift_query_read(q, schema="secure")
q = "select count(id) as sponsors_updated_status, date_trunc('month', original_timestamp) as month from sponsors_updated_item_status group by month order by month desc"
sponsors_updated_status = redshift_query_read(q, schema="secure")
q = "select count(id) as sponsors_saved, date_trunc('month', original_timestamp) as month from sponsor_saved_item_settings group by month order by month desc"
sponsors_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as sponsors_deleted, date_trunc('month', original_timestamp) as month from sponsor_deleted_item group by month order by month desc"
sponsors_deleted = redshift_query_read(q, schema="secure")

q = "select count(id) as sponsorscategory_deleted, date_trunc('month', original_timestamp) as month from sponsor_categories_deleted_item group by month order by month desc"
sponsorscategory_deleted = redshift_query_read(q, schema="secure")
q = "select count(id) as sponsorscategory_saved, date_trunc('month', original_timestamp) as month from sponsor_categories_saved_item_settings group by month order by month desc"
sponsorscategory_saved = redshift_query_read(q, schema="secure")
q = "select count(id) as sponsorscategory_started_edit, date_trunc('month', original_timestamp) as month from sponsor_categories_started_item_edit group by month order by month desc"
sponsorscategory_started_edit = redshift_query_read(q, schema="secure")

In [56]:
len(sponsors_started_edit), len(sponsors_updated_status), len(sponsors_saved), len(sponsors_deleted), len(sponsorscategory_deleted), len(sponsorscategory_saved), len(sponsorscategory_started_edit)

(2, 2, 1, 1, 2, 2, 2)

In [81]:
cols = ["month", "sponsors_started_edit", "sponsors_updated_status", "sponsors_saved",
        "sponsors_deleted", "sponsorscategory_deleted", "sponsorscategory_saved", 
        "sponsorscategory_started_edit"]
sponsors_started_edit.merge(sponsors_updated_status, on="month", how="outer").merge(sponsors_saved, on="month", how="outer").merge(sponsors_deleted, on="month", how="outer").merge(sponsorscategory_deleted, on="month", how="outer").merge(sponsorscategory_saved, on="month", how="outer").merge(sponsorscategory_started_edit, on="month", how="outer")[cols].sort_values('month', ascending=True).fillna(0)

,month,sponsors_started_edit,sponsors_updated_status,sponsors_saved,sponsors_deleted,sponsorscategory_deleted,sponsorscategory_saved,sponsorscategory_started_edit
1,2020-03-01,11,10,7.0,1.0,2,81,47
0,2020-04-01,9,2,0.0,0.0,14,47,33


# mappings

In [61]:
q = "select count(id) as mappings_created, date_trunc('month', original_timestamp) as month from created_data_mapping group by month order by month desc"
mappings_created = redshift_query_read(q, schema="secure")
q = "select count(id) as mappings_updated, date_trunc('month', original_timestamp) as month from updated_mappings group by month order by month desc"
mappings_updated = redshift_query_read(q, schema="secure")

In [83]:
cols = ["month", "mappings_created", "mappings_updated"]
mappings_created.merge(mappings_updated, on="month", how="outer")[cols].sort_values('month', ascending=True).fillna(0)

,month,mappings_created,mappings_updated
3,2020-01-01,47,80
2,2020-02-01,43,318
1,2020-03-01,106,314
0,2020-04-01,14,149


In [4]:
q = '''select t.table_name
        from information_schema.tables t
        where t.table_schema = 'secure'  -- put schema name here
              and t.table_type = 'BASE TABLE'
        order by t.table_name;'''
tables = redshift_query_read(q, schema="secure")

In [59]:
tables[tables['table_name'].str.contains('map')]

,table_name
355,created_data_mapping
1071,updated_mappings
